In [48]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import random
import math
import time
from genetic_matching import genetic_matching_with_elite

Daten laden und vorbereiten

In [49]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [50]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

print(f"Loaded: {len(personen_df)} people, {len(teilaufgaben_df)} tasks")
print(f"Total workload: {teilaufgaben_df['aufwand'].sum():.1f} hours")

Loaded: 150 people, 8 tasks
Total workload: 19.5 hours


Initialisierung

In [51]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [52]:
# Liste zur Speicherung der Zuweisungsergebnisse
matching_ergebnisse = []

In [53]:
# Run Hungarian Algorithm
print("\n" + "="*50)
print("GENETIC ALGORITHM MATCHING")
print("="*50)


GENETIC ALGORITHM MATCHING


In [54]:
genetic_results, genetic_time = genetic_matching_with_elite(teilaufgaben_df, personen_df)


=== Enhanced Genetic Algorithm ===

Trying parameter set 1/3

=== Genetic Algorithm ===
Population size: 50
Generations: 100
Tasks: 8, People: 150
Generation 0: Best=-7020.5, Avg=-8784.8
Generation 20: Best=-6538.8, Avg=-6700.8
Generation 40: Best=-6136.3, Avg=-6519.6
Generation 60: Best=-6136.3, Avg=-6376.0
Generation 80: Best=-5031.1, Avg=-5933.0
Generation 99: Best=-5031.1, Avg=-5486.9
Execution time: 3.660 seconds
Best fitness: -5031.1
Found assignments: 8

Trying parameter set 2/3

=== Genetic Algorithm ===
Population size: 100
Generations: 150
Tasks: 8, People: 150
Generation 0: Best=-7020.5, Avg=-8793.1
Generation 20: Best=-6164.1, Avg=-6256.8
Generation 40: Best=-5682.4, Avg=-5883.6
Generation 60: Best=-5182.7, Avg=-5299.2
Generation 80: Best=-4577.2, Avg=-4780.0
Generation 100: Best=-4577.2, Avg=-4779.4
Generation 120: Best=-4577.2, Avg=-4808.8
Generation 140: Best=-4577.2, Avg=-4741.5
Generation 149: Best=-4577.2, Avg=-4798.9
Execution time: 10.119 seconds
Best fitness: -457

In [55]:
for result in genetic_results:
    matching_ergebnisse.append({
        "teilaufgabe_id": result["teilaufgabe_id"],
        "teilaufgabe": result["teilaufgabe"],
        "projekt_id": result["projekt_id"],
        "kompetenz": result["kompetenz"],
        "person_id": result["person_id"],
        "name": result["name"],
        "zugewiesener_aufwand": result["zugewiesener_aufwand"]
    })

print(f"\nGenetic Algorithm Results:")
print(f"Found {len(matching_ergebnisse)} assignments")
print(f"Execution time: {genetic_time:.3f} seconds")


Genetic Algorithm Results:
Found 8 assignments
Execution time: 24.023 seconds


In [56]:
# Display results summary
print(f"\n" + "="*50)
print("RESULTS SUMMARY")
print("="*50)
print(f"Total assignments: {len(matching_ergebnisse)}")
print(f"Total workload assigned: {sum(item['zugewiesener_aufwand'] for item in matching_ergebnisse):.1f} hours")
print(f"Total workload needed: {teilaufgaben_df['aufwand'].sum():.1f} hours")


RESULTS SUMMARY
Total assignments: 8
Total workload assigned: 19.5 hours
Total workload needed: 19.5 hours


In [57]:
# Show assignments by task
print(f"\nAssignments by task:")
for _, ta in teilaufgaben_df.iterrows():
    task_assignments = [item for item in matching_ergebnisse if item['teilaufgabe_id'] == ta['teilaufgabe_id']]
    assigned_workload = sum(item['zugewiesener_aufwand'] for item in task_assignments)
    print(f"  - {ta['bezeichnung']}: {assigned_workload:.1f}/{ta['aufwand']:.1f} hours assigned")


Assignments by task:
  - Entwurf CAD-Baugruppe: 3.0/3.0 hours assigned
  - Simulation FEM-Struktur: 2.5/2.5 hours assigned
  - Signalverarbeitung Sensor: 2.0/2.0 hours assigned
  - Entwicklung Python-Modul: 3.5/3.5 hours assigned
  - Statistische Auswertung: 2.0/2.0 hours assigned
  - Optische Analyse: 1.5/1.5 hours assigned
  - Datenbankmodellierung: 2.0/2.0 hours assigned
  - Thermodynamische Bewertung: 3.0/3.0 hours assigned


Ergebnisse speichern

In [58]:
# Save results
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print(f"\nMatching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")


Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
